<a href="https://colab.research.google.com/github/OphanimLLC/pytvlwcharts/blob/main/new_main_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Oct 28 15:18:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Default title text
!pip install tk
!pip install turtle

!pip install --quiet gym_anytrading
!pip install --quiet alpaca
!pip install --quiet alpaca_trade_api
!pip install --quiet stable_baselines3
!pip install --quiet stockstats
!pip install --quiet yfinance
!pip install --quiet pyfolio
!pip install --quiet --upgrade git+https://github.com/TechfaneTechnologies/pytvlwcharts.git

#!pip install 
#!pip install 

# Gym stuff
import gym
import gym_anytrading
from gym import Env
from gym.utils import seeding
from gym import spaces

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for turtle: filename=turtle-0.0.1-py3-none-any.whl size=22217 sha256=4331f8a3abb38e5268413bc854485cf6981174553d8516fa2f2ed09e78f69744
  Stored in directory: /root/.cache/pip/wheels/b7/bc/0f/7882965978d4fbe3d255285b3dc90ad7217b90199569bca0d0
Successfully built turtle
     |████████████████████████████████| 171 kB 32.5 MB/s 
     |████████████████████████████████| 106 kB 52.3 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 299 kB 70.6 MB/s 
     |████████████████████████████████| 1.1 MB 81.6 MB/s 
     |████████████████████████████████| 170 kB 37.7 MB/s 
     |████████████████████████████████| 1.5 MB 95.9 MB/s 
     |████████████████████████████████| 62 kB 916 kB/s 
     |████████████████████████████████| 91 kB 9.7 MB/s 
   

In [ ]:
# Gym stuff
import gym
# import gym_anytrading
from gym import Env
from gym.utils import seeding
from gym import spaces

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# from alpaca_trade_api.rest import REST, TimeFrame

from datetime import date
import datetime
import time

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
# from stable_baselines3.common.vec_env import SubprocVecEnv
# from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import A2C, PPO, DDPG, DQN, SAC, TD3  #There are models available in the stable baslines package and will try them in the meantime
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [ ]:
import pickle
import warnings
warnings.filterwarnings('ignore')

from stockstats import StockDataFrame as Sdf
import yfinance as yf
import pyfolio

In [ ]:
SYMBOL = 'SPY' #@param {type:"string"}
    #@markdown Input the start date and end date in this format '2021-11-01' (YYYY-MM-DD)
Start = "2000-01-01" #@param {type:"string"}
#@markdown Input the horse ID in this format '134731 129309' (add only a space between IDs)
End = "2022-10-27" #@param {type:"string"}

#@markdown If you want to train the model with with most recent data fill the below for days or keep it as "none" to use above date
Days = "none" #@param {type:"string"}

#@title ALPACA Keys

#@markdown Enter the  **Key ID**
# Key = "PKG9PVMJP3W63LRY153J" #@param {type:"string"}
Key = "AKOCVSLHSMCUA2SP7JLP"

#@markdown Enter the **Secret Key**
# Secret = "bhL3qnhT4AC6hCdSc1GyIEzd84uRWW287RWTdt8E" #@param {type:"string"}
Secret = "1yT1t2VzRhpOYUBjhS5TIOn0XT0z0GD885S3eGcM" #@param {type:"string"}

In [ ]:
def calculate_price(df):
    data = df.copy()
    data = data[['datadate', 'tic', 'adjcp', 'open', 'high', 'low', 'volume']]
    data = data.sort_values(['datadate'], ignore_index=True)
    return data

In [ ]:
def add_technical_indicator(df):
    stock = Sdf.retype(df.copy())
    stock['close'] = stock['adjcp']
    unique_ticker = stock.tic.unique()
    macd = pd.DataFrame()
    rsi = pd.DataFrame()
    cci = pd.DataFrame()
    dx = pd.DataFrame()

    for i in range(len(unique_ticker)):
        ## macd
        temp_macd = stock[stock.tic == unique_ticker[i]]['macd']
        temp_macd = pd.DataFrame(temp_macd)
        macd = macd.append(temp_macd, ignore_index=True)
        ## rsi
        temp_rsi = stock[stock.tic == unique_ticker[i]]['rsi_30']
        temp_rsi = pd.DataFrame(temp_rsi)
        rsi = rsi.append(temp_rsi, ignore_index=True)
        ## cci
        temp_cci = stock[stock.tic == unique_ticker[i]]['cci_30']
        temp_cci = pd.DataFrame(temp_cci)
        cci = cci.append(temp_cci, ignore_index=True)
        ## adx
        temp_dx = stock[stock.tic == unique_ticker[i]]['dx_30']
        temp_dx = pd.DataFrame(temp_dx)
        dx = dx.append(temp_dx, ignore_index=True)


    df['macd'] = macd
    df['rsi'] = rsi
    df['cci'] = cci
    df['adx'] = dx

    return df

In [ ]:
def load_dataset():
    if Days == "none":
        start_date = Start
        end_date = End
    else:
        today = datetime.datetime.now()
        DD = datetime.timedelta(days=int(Days))
        earlier = today - DD

        earlier_str = earlier.strftime("%Y%m%d")
        today_str = today.strftime("%Y%m%d")

        start_date = str(earlier)[0:10]
        end_date = today.date().strftime("%Y-%m-%d")

    # Printing the selcted data
    print("The model will train from {start} to {end} dates".format(start=start_date, end=end_date))

    KEY_ID = Key
    SECRET_KEY = Secret
    
    api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://api.alpaca.markets")
    
    df = api.get_bars(SYMBOL,"1Day", start_date, end_date, adjustment='all', feed='sip').df
    df.drop("trade_count", axis=1, inplace=True)
    df.drop("vwap", axis=1, inplace=True)
    df['adjcp'] = df['close']
    df['tic'] = SYMBOL
    df = df.dropna()
    df["datadate"] = pd.to_datetime(df.index)
    df = calculate_price(df)
    df = add_technical_indicator(df)
    df.fillna(method='bfill',inplace=True)
    return df


In [ ]:
# df = load_dataset()
# df

In [ ]:
# df.to_csv("main_data.csv", index=False)

In [ ]:
# unique_date = df.datadate.unique
# display(unique_date)
# print("Length of unique date is: ", len(unique_date))

In [ ]:
def calculate_turbulence(df):
    print("Entered")
    df_price_pivot=df.pivot(index='datadate', columns='tic', values='adjcp')
    print("Done Pivot")
    unique_date = df.datadate.unique()
    print("Length of unique date is: ", len(unique_date))
    # start after a year
    start = 253
    turbulence_index = [0]*start
    #turbulence_index = [0]
    count = 0
    for i in range(start,len(unique_date)):
        current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
        hist_price = df_price_pivot[[n in unique_date[0:i] for n in df_price_pivot.index]]
        cov_temp = hist_price.cov()
        current_temp=(current_price - np.mean(hist_price,axis=0))
        temp = current_temp.values.dot(np.linalg.inv(cov_temp)).dot(current_temp.values.T)
        if temp>0:
            count+=1
            if count>2:
                turbulence_temp = temp[0][0]
            else:
                #avoid large outlier because of the calculation just begins
                turbulence_temp=0
        else:
            turbulence_temp=0
        turbulence_index.append(turbulence_temp)
#         print("Iterations left: ", len(unique_date) - i)      
    turbulence_index = pd.DataFrame({'datadate':df_price_pivot.index, 'turbulence':turbulence_index})
    return turbulence_index

In [ ]:
def add_turbulence(df):
    turbulence_index = calculate_turbulence(df)
    df = df.merge(turbulence_index, on='datadate')
    df = df.sort_values(['datadate']).reset_index(drop=True)
    return df

In [ ]:
# processed_df = df.copy()
# processed_df

In [ ]:
# df = add_turbulence(df)
# df

In [ ]:
# df.to_csv("data/turbulence_data.csv", index=False)

In [ ]:
df = pd.read_csv("turbulence_data.csv")
df

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:04:00+00:00,SPY,186.06,186.06,186.06,186.06,470,0.000000,0.000000,-66.666667,100.000000,0.000000
1,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.000000
2,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.000000
3,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.000000
4,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
57995,2016-03-14 11:07:00+00:00,SPY,180.74,180.74,180.74,180.74,301,-0.018151,44.625136,-72.477064,21.824283,0.776241
57996,2016-03-14 11:08:00+00:00,SPY,180.72,180.76,180.76,180.72,3120,-0.020913,43.718166,-90.126292,26.609318,0.748927
57997,2016-03-14 11:09:00+00:00,SPY,180.73,180.72,180.73,180.71,5420,-0.022040,44.303675,-118.004866,28.934235,0.759701
57998,2016-03-14 11:10:00+00:00,SPY,180.73,180.72,180.73,180.72,9550,-0.022673,44.303675,-116.182573,28.934235,0.757751


In [ ]:
df['datadate']= pd.to_datetime(df['datadate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58000 entries, 0 to 57999
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   datadate    58000 non-null  datetime64[ns, UTC]
 1   tic         58000 non-null  object             
 2   adjcp       58000 non-null  float64            
 3   open        58000 non-null  float64            
 4   high        58000 non-null  float64            
 5   low         58000 non-null  float64            
 6   volume      58000 non-null  int64              
 7   macd        58000 non-null  float64            
 8   rsi         58000 non-null  float64            
 9   cci         58000 non-null  float64            
 10  adx         58000 non-null  float64            
 11  turbulence  58000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(9), int64(1), object(1)
memory usage: 5.3+ MB


In [ ]:
percentage = 60

train_data = df[1:int(len(df)*percentage/100)]
test_data = df[(int(len(df)*percentage/100)+1):len(df)]


In [ ]:
train_data

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
1,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
2,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
3,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
4,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
5,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
34795,2016-02-02 12:59:00+00:00,SPY,172.10,172.03,172.10,172.03,17305,0.029398,56.470884,189.531448,17.517758,0.0
34796,2016-02-02 13:00:00+00:00,SPY,172.15,172.13,172.19,171.83,48448,0.040178,58.161466,159.663866,16.537462,0.0
34797,2016-02-02 13:01:00+00:00,SPY,172.13,172.15,172.16,172.09,7020,0.046571,57.241544,209.414605,16.537462,0.0
34798,2016-02-02 13:02:00+00:00,SPY,172.19,172.10,172.19,172.10,7721,0.055835,59.242193,207.435572,11.022895,0.0


In [ ]:
test_data

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
34801,2016-02-02 13:05:00+00:00,SPY,172.04,172.45,172.45,172.04,49080,0.053083,52.556310,154.365733,23.979458,0.000000
34802,2016-02-02 13:06:00+00:00,SPY,172.06,172.08,172.10,172.06,3300,0.047938,53.296367,71.029400,23.979458,0.000000
34803,2016-02-02 13:07:00+00:00,SPY,172.04,172.06,172.06,172.04,5425,0.041765,52.450007,47.189734,20.757103,0.000000
34804,2016-02-02 13:08:00+00:00,SPY,172.08,172.08,172.09,172.08,850,0.039643,53.962599,68.274401,23.927448,0.000000
34805,2016-02-02 13:09:00+00:00,SPY,172.07,172.07,172.08,172.07,9360,0.036732,53.522278,56.511057,22.185607,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
57995,2016-03-14 11:07:00+00:00,SPY,180.74,180.74,180.74,180.74,301,-0.018151,44.625136,-72.477064,21.824283,0.776241
57996,2016-03-14 11:08:00+00:00,SPY,180.72,180.76,180.76,180.72,3120,-0.020913,43.718166,-90.126292,26.609318,0.748927
57997,2016-03-14 11:09:00+00:00,SPY,180.73,180.72,180.73,180.71,5420,-0.022040,44.303675,-118.004866,28.934235,0.759701
57998,2016-03-14 11:10:00+00:00,SPY,180.73,180.72,180.73,180.72,9550,-0.022673,44.303675,-116.182573,28.934235,0.757751


### Setting Variables

In [ ]:
# unique_trade_date = df[(df.datadate >= "2015-12-02") & (df.datadate <= "2022-09-06")].datadate.unique()
unique_trade_date = test_data.datadate.unique()
unique_trade_date

<DatetimeArray>
['2016-02-02 13:05:00+00:00', '2016-02-02 13:06:00+00:00',
 '2016-02-02 13:07:00+00:00', '2016-02-02 13:08:00+00:00',
 '2016-02-02 13:09:00+00:00', '2016-02-02 13:10:00+00:00',
 '2016-02-02 13:11:00+00:00', '2016-02-02 13:12:00+00:00',
 '2016-02-02 13:13:00+00:00', '2016-02-02 13:14:00+00:00',
 ...
 '2016-03-14 11:01:00+00:00', '2016-03-14 11:03:00+00:00',
 '2016-03-14 11:04:00+00:00', '2016-03-14 11:05:00+00:00',
 '2016-03-14 11:06:00+00:00', '2016-03-14 11:07:00+00:00',
 '2016-03-14 11:08:00+00:00', '2016-03-14 11:09:00+00:00',
 '2016-03-14 11:10:00+00:00', '2016-03-14 11:12:00+00:00']
Length: 23199, dtype: datetime64[ns, UTC]

In [ ]:
# 100 shares per trade
MAX_NORMALIZE = 100
# initial amount of money we have in account
INITIAL_ACC_BAL=1000000
# total number of stocks in  portfolio
TOTAL_STOCKS = 1
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001
REWARD_SCALING = 1e-4

# Training Gym environment
1. Define action_space and obeservation_space in the Environment's Constructor
2. Reset Method - reset environment
3. Define Sell Stock Method
4. Define Buy Stock Method
5. Define Step function

In [ ]:
from turtle import Shape
import gym
# import gym_anytrading
from gym import Env
from gym.utils import seeding
from gym import spaces

import numpy as np
import pandas as pd

#Reference https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e
# https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020

# 100 shares per trade
MAX_NORMALIZE = 100
# initial amount of money we have in account
INITIAL_ACC_BAL=1000000
# total number of stocks in  portfolio
TOTAL_STOCKS = 1
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001
REWARD_SCALING = 1e-4
SHAPE = 152


class StockTrainEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df, day = 0):
        self.day = day
        print("Day is: ", day)
        self.df = df

        # action_space 
        self.action_space = spaces.Box(low = -1, high = 1,shape = (TOTAL_STOCKS,)) 
        
        # Shape = 181: [Current Balance]+[prices 1-30]+[owned shares 1-30] 
        # +[macd 1-30]+ [rsi 1-30] + [cci 1-30] + [adx 1-30]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (SHAPE,))
        
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        print('**********')
        print("Length is: ", len(self.data))
        print("Shape is: ", self.data.shape)
        display(self.data)
        print('**********')
        self.terminal = False             
        
        # initalize state
        self.state = np.array([INITIAL_ACC_BAL] + \
                      self.data.adjcp.values[self.day:self.day+30].tolist() + \
                      [0]*TOTAL_STOCKS + \
                      self.data.macd.values[self.day:self.day+30].tolist() + \
                      self.data.rsi.values[self.day:self.day+30].tolist() + \
                      self.data.cci.values[self.day:self.day+30].tolist() + \
                      self.data.adx.values[self.day:self.day+30].tolist())
        # initialize reward
        self.reward = 0
        self.cost = 0
        # memorize all the total balance change
        self.asset_memory = [INITIAL_ACC_BAL]
        self.rewards_memory = []
        self.trades = 0
        #self.reset()

        self._seed()


    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.state[index+TOTAL_STOCKS+1] > 0:
            #update balance
            self.state[0] += \
            self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
             (1- TRANSACTION_FEE_PERCENT)

            self.state[index+TOTAL_STOCKS+1] -= min(abs(action), self.state[index+TOTAL_STOCKS+1])
            self.cost +=self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
             TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            pass

    
    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        available_amount = self.state[0] // self.state[index+1]
        # print('available_amount:{}'.format(available_amount))

        #update balance
        self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                          (1+ TRANSACTION_FEE_PERCENT)

        self.state[index+TOTAL_STOCKS+1] += min(available_amount, action)

        self.cost+=self.state[index+1]*min(available_amount, action)* \
                          TRANSACTION_FEE_PERCENT
        self.trades+=1
        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            plt.savefig('account_value_train.png')
            plt.close()
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            
            #print("end_total_asset:{}".format(end_total_asset))
            df_total_value = pd.DataFrame(self.asset_memory)
            df_total_value.to_csv('account_value_train.csv')
            #print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):61]))- INITIAL_ACCOUNT_BALANCE ))
            #print("total_cost: ", self.cost)
            #print("total_trades: ", self.trades)
            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)
            sharpe = (252**0.5)*df_total_value['daily_return'].mean()/ \
                  df_total_value['daily_return'].std()
            #print("Sharpe: ",sharpe)
            #print("=================================")
            df_rewards = pd.DataFrame(self.rewards_memory)
            #df_rewards.to_csv('account_rewards_train.csv')
            
            # print('total asset: {}'.format(self.state[0]+ sum(np.array(self.state[1:29])*np.array(self.state[29:]))))
            #with open('obs.pkl', 'wb') as f:  
            #    pickle.dump(self.state, f)
            
            return self.state, self.reward, self.terminal,{}

        else:
            # print(np.array(self.state[1:29]))

            actions = actions * MAX_NORMALIZE
            #actions = (actions.astype(int))
            
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            #print("begin_total_asset:{}".format(begin_total_asset))
            
            argsort_actions = np.argsort(actions)
            
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            self.day += 1
            self.data = self.df.loc[self.day,:]         
            #load next state
            # print("stock_shares:{}".format(self.state[29:]))
            self.state =  np.array([self.state[0]] + \
                    self.data.adjcp.values[self.day:self.day+30].tolist() + \
                    list(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]) + \
                    self.data.macd.values[self.day:self.day+30].tolist() + \
                    self.data.rsi.values[self.day:self.day+30].tolist() + \
                    self.data.cci.values[self.day:self.day+30].tolist() + \
                    self.data.adx.values[self.day:self.day+30].tolist())
            
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            self.asset_memory.append(end_total_asset)
            #print("end_total_asset:{}".format(end_total_asset))
            
            self.reward = end_total_asset - begin_total_asset            
            # print("step_reward:{}".format(self.reward))
            self.rewards_memory.append(self.reward)
            
            self.reward = self.reward*REWARD_SCALING



        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [INITIAL_ACC_BAL]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        self.cost = 0
        self.trades = 0
        self.terminal = False 
        self.rewards_memory = []
        #initiate state
        self.state = np.array([INITIAL_ACC_BAL] + \
                      self.data.adjcp.values[self.day:self.day+30].tolist() + \
                      [0]*TOTAL_STOCKS + \
                      self.data.macd.values[self.day:self.day+30].tolist() + \
                      self.data.rsi.values[self.day:self.day+30].tolist() + \
                      self.data.cci.values[self.day:self.day+30].tolist() + \
                      self.data.adx.values[self.day:self.day+30].tolist()) 
        # iteration += 1 
        return self.state
    
    def render(self, mode='human'):
        return self.state

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

class StockValidationEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df, day = 0, turbulence_threshold=140, iteration=''):
        self.day = day
        self.df = df
        
        # action_space normalization and shape is TOTAL_STOCKS
        self.action_space = spaces.Box(low = -1, high = 1,shape = (TOTAL_STOCKS,)) 
        
        # Shape = 181: [Current Balance]+[prices 1-30]+[owned shares 1-30] 
        # +[macd 1-30]+ [rsi 1-30] + [cci 1-30] + [adx 1-30]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (SHAPE,))
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold
        
        # initalize state
        self.state = np.array([INITIAL_ACC_BAL] + \
                      self.data.adjcp.values[self.day:self.day+30].tolist() + \
                      [0]*TOTAL_STOCKS + \
                      self.data.macd.values[self.day:self.day+30].tolist() + \
                      self.data.rsi.values[self.day:self.day+30].tolist() + \
                      self.data.cci.values[self.day:self.day+30].tolist() + \
                      self.data.adx.values[self.day:self.day+30].tolist())
        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        # memorize all the total balance change
        self.asset_memory = [INITIAL_ACC_BAL]
        self.rewards_memory = []
        #self.reset()
        self._seed()
        
        self.iteration=iteration


    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.turbulence<self.turbulence_threshold:
            if self.state[index+TOTAL_STOCKS+1] > 0:
                #update balance
                self.state[0] += \
                self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
                 (1- TRANSACTION_FEE_PERCENT)
                
                self.state[index+TOTAL_STOCKS+1] -= min(abs(action), self.state[index+TOTAL_STOCKS+1])
                self.cost +=self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
                 TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
        else:
            # if turbulence goes over threshold, just clear out all positions 
            if self.state[index+TOTAL_STOCKS+1] > 0:
                #update balance
                self.state[0] += self.state[index+1]*self.state[index+TOTAL_STOCKS+1]* \
                              (1- TRANSACTION_FEE_PERCENT)
                self.state[index+TOTAL_STOCKS+1] =0
                self.cost += self.state[index+1]*self.state[index+TOTAL_STOCKS+1]* \
                              TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
    
    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        if self.turbulence< self.turbulence_threshold:
            available_amount = self.state[0] // self.state[index+1]
            # print('available_amount:{}'.format(available_amount))
            
            #update balance
            self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                              (1+ TRANSACTION_FEE_PERCENT)

            self.state[index+TOTAL_STOCKS+1] += min(available_amount, action)
            
            self.cost+=self.state[index+1]*min(available_amount, action)* \
                              TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            # if turbulence goes over threshold, just stop buying
            pass
        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            plt.savefig('account_value_validation_{}.png'.format(self.iteration))
            plt.close()
            df_total_value = pd.DataFrame(self.asset_memory)
            df_total_value.to_csv('account_value_validation_{}.csv'.format(self.iteration))
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            #print("previous_total_asset:{}".format(self.asset_memory[0]))           

            #print("end_total_asset:{}".format(end_total_asset))
            #print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):61]))- self.asset_memory[0] ))
            #print("total_cost: ", self.cost)
            #print("total trades: ", self.trades)

            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)
            sharpe = (4**0.5)*df_total_value['daily_return'].mean()/ \
                  df_total_value['daily_return'].std()
            #print("Sharpe: ",sharpe)
            
            #df_rewards = pd.DataFrame(self.rewards_memory)
            #df_rewards.to_csv('account_rewards_trade_{}.csv'.format(self.iteration))
            
            # print('total asset: {}'.format(self.state[0]+ sum(np.array(self.state[1:29])*np.array(self.state[29:]))))
            #with open('obs.pkl', 'wb') as f:  
            #    pickle.dump(self.state, f)
            
            return self.state, self.reward, self.terminal,{}

        else:
            # print(np.array(self.state[1:29]))

            actions = actions * MAX_NORMALIZE
            #actions = (actions.astype(int))
            if self.turbulence>=self.turbulence_threshold:
                actions=np.array([-MAX_NORMALIZE]*TOTAL_STOCKS)
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            #print("begin_total_asset:{}".format(begin_total_asset))
            
            argsort_actions = np.argsort(actions)
            
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            self.day += 1
            self.data = self.df.loc[self.day,:]         
            self.turbulence = self.data['turbulence'].values[0]
            #print(self.turbulence)
            #load next state
            # print("stock_shares:{}".format(self.state[29:]))
            self.state =  np.array([self.state[0]] + \
                    self.data.adjcp.values[self.day:self.day+30].tolist() + \
                    list(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]) + \
                    self.data.macd.values[self.day:self.day+30].tolist() + \
                    self.data.rsi.values[self.day:self.day+30].tolist() + \
                    self.data.cci.values[self.day:self.day+30].tolist() + \
                    self.data.adx.values[self.day:self.day+30].tolist())
            
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            self.asset_memory.append(end_total_asset)
            #print("end_total_asset:{}".format(end_total_asset))
            
            self.reward = end_total_asset - begin_total_asset            
            # print("step_reward:{}".format(self.reward))
            self.rewards_memory.append(self.reward)
            
            self.reward = self.reward*REWARD_SCALING

        return self.state, self.reward, self.terminal, {}

    def reset(self):  
        self.asset_memory = [INITIAL_ACC_BAL]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        print("HEYYYY")
        print("******************\n", self.data, "******************\n")
        print("\n\nShape is: ", self.data.shape, "\n\n")
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.terminal = False 
        #self.iteration=self.iteration
        self.rewards_memory = []
        #initiate state
        self.state = np.array([INITIAL_ACC_BAL] + \
                      self.data.adjcp.values[self.day:self.day+30].tolist() + \
                      [0]*TOTAL_STOCKS + \
                      self.data.macd.values[self.day:self.day+30].tolist() + \
                      self.data.rsi.values[self.day:self.day+30].tolist()  + \
                      self.data.cci.values[self.day:self.day+30].tolist()  + \
                      self.data.adx.values[self.day:self.day+30].tolist()) 
            
        return self.state
    
    def render(self, mode='human',close=False):
        return self.state
    

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]


class StockTradeEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df,day = 0,turbulence_threshold=140
                 ,initial=True, previous_state=[], model_name='', iteration=''):
        #money = 10 , scope = 1
        self.day = day
        self.df = df
        self.initial = initial
        self.previous_state = previous_state
        
        # action_space normalization and shape is TOTAL_STOCKS
        self.action_space = spaces.Box(low = -1, high = 1,shape = (TOTAL_STOCKS,)) 
        
        # Shape = 181: [Current Balance]+[prices 1-30]+[owned shares 1-30] 
        # +[macd 1-30]+ [rsi 1-30] + [cci 1-30] + [adx 1-30]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (SHAPE,))
        
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold
        
        # initalize state
        self.state = np.array([INITIAL_ACC_BAL] + \
                      self.data.adjcp.values[self.day:self.day+30].tolist() + \
                      [0]*TOTAL_STOCKS + \
                      self.data.macd.values[self.day:self.day+30].tolist() + \
                      self.data.rsi.values[self.day:self.day+30].tolist() + \
                      self.data.cci.values[self.day:self.day+30].tolist() + \
                      self.data.adx.values[self.day:self.day+30].tolist())
        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        # memorize all the total balance change
        self.asset_memory = [INITIAL_ACC_BAL]
        self.rewards_memory = []
        #self.reset()
        self._seed()
        self.model_name=model_name        
        self.iteration=iteration


    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.turbulence<self.turbulence_threshold:
            if self.state[index+TOTAL_STOCKS+1] > 0:
                #update balance
                self.state[0] += \
                self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
                 (1- TRANSACTION_FEE_PERCENT)
                
                self.state[index+TOTAL_STOCKS+1] -= min(abs(action), self.state[index+TOTAL_STOCKS+1])
                self.cost +=self.state[index+1]*min(abs(action),self.state[index+TOTAL_STOCKS+1]) * \
                 TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
        else:
            # if turbulence goes over threshold, just clear out all positions 
            if self.state[index+TOTAL_STOCKS+1] > 0:
                #update balance
                self.state[0] += self.state[index+1]*self.state[index+TOTAL_STOCKS+1]* \
                              (1- TRANSACTION_FEE_PERCENT)
                self.state[index+TOTAL_STOCKS+1] =0
                self.cost += self.state[index+1]*self.state[index+TOTAL_STOCKS+1]* \
                              TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
    
    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        if self.turbulence< self.turbulence_threshold:
            available_amount = self.state[0] // self.state[index+1]
            # print('available_amount:{}'.format(available_amount))
            
            #update balance
            self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                              (1+ TRANSACTION_FEE_PERCENT)

            self.state[index+TOTAL_STOCKS+1] += min(available_amount, action)
            
            self.cost+=self.state[index+1]*min(available_amount, action)* \
                              TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            # if turbulence goes over threshold, just stop buying
            pass
        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            plt.savefig('account_value_trade_{}_{}.png'.format(self.model_name, self.iteration))
            plt.close()
            df_total_value = pd.DataFrame(self.asset_memory)
            df_total_value.to_csv('account_value_trade_{}_{}.csv'.format(self.model_name, self.iteration))
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            print("previous_total_asset:{}".format(self.asset_memory[0]))           

            print("end_total_asset:{}".format(end_total_asset))
            print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))- self.asset_memory[0] ))
            print("total_cost: ", self.cost)
            print("total trades: ", self.trades)

            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)
            sharpe = (4**0.5)*df_total_value['daily_return'].mean()/ \
                  df_total_value['daily_return'].std()
            print("Sharpe: ",sharpe)
            
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.to_csv('account_rewards_trade_{}_{}.csv'.format(self.model_name, self.iteration))
            
            # print('total asset: {}'.format(self.state[0]+ sum(np.array(self.state[1:29])*np.array(self.state[29:]))))
            #with open('obs.pkl', 'wb') as f:  
            #    pickle.dump(self.state, f)
            
            return self.state, self.reward, self.terminal,{}

        else:
            # print(np.array(self.state[1:29]))

            actions = actions * MAX_NORMALIZE
            #actions = (actions.astype(int))
            if self.turbulence>=self.turbulence_threshold:
                actions=np.array([-MAX_NORMALIZE]*TOTAL_STOCKS)
                
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            #print("begin_total_asset:{}".format(begin_total_asset))
            
            argsort_actions = np.argsort(actions)
            
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            self.day += 1
            self.data = self.df.loc[self.day,:]         
            self.turbulence = self.data['turbulence'].values[0]
            #print(self.turbulence)
            #load next state
            # print("stock_shares:{}".format(self.state[29:]))
            self.state =  np.array([self.state[0]] + \
                    self.data.adjcp.values[self.day:self.day+30].tolist() + \
                    list(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]) + \
                    self.data.macd.values[self.day:self.day+30].tolist() + \
                    self.data.rsi.values[self.day:self.day+30].tolist() + \
                    self.data.cci.values[self.day:self.day+30].tolist() + \
                    self.data.adx.values[self.day:self.day+30].tolist())
            
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(TOTAL_STOCKS+1)])*np.array(self.state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            self.asset_memory.append(end_total_asset)
            #print("end_total_asset:{}".format(end_total_asset))
            
            self.reward = end_total_asset - begin_total_asset            
            # print("step_reward:{}".format(self.reward))
            self.rewards_memory.append(self.reward)
            
            self.reward = self.reward*REWARD_SCALING


        return self.state, self.reward, self.terminal, {}

    def reset(self):  
        if self.initial:
            self.asset_memory = [INITIAL_ACC_BAL]
            self.day = 0
            self.data = self.df.loc[self.day,:]
            self.turbulence = 0
            self.cost = 0
            self.trades = 0
            self.terminal = False 
            #self.iteration=self.iteration
            self.rewards_memory = []
            #initiate state
            self.state = np.array([INITIAL_ACC_BAL] + \
                          self.data.adjcp.values[self.day:self.day+30].tolist() + \
                          [0]*TOTAL_STOCKS + \
                          self.data.macd.values[self.day:self.day+30].tolist() + \
                          self.data.rsi.values[self.day:self.day+30].tolist()  + \
                          self.data.cci.values[self.day:self.day+30].tolist()  + \
                          self.data.adx.values[self.day:self.day+30].tolist()) 
        else:
            previous_total_asset = self.previous_state[0]+ \
            sum(np.array(self.previous_state[1:(TOTAL_STOCKS+1)])*np.array(self.previous_state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]))
            self.asset_memory = [previous_total_asset]
            #self.asset_memory = [self.previous_state[0]]
            self.day = 0
            self.data = self.df.loc[self.day,:]
            self.turbulence = 0
            self.cost = 0
            self.trades = 0
            self.terminal = False 
            #self.iteration=iteration
            self.rewards_memory = []
            #initiate state
            #self.previous_state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)]
            #[0]*TOTAL_STOCKS + \

            self.state = np.array([self.previous_state[0]] + \
                          self.data.adjcp.values[self.day:self.day+30].tolist() + \
                          list(self.previous_state[(TOTAL_STOCKS+1):(TOTAL_STOCKS*2+1)])+ \
                          self.data.macd.values[self.day:self.day+30].tolist() + \
                          self.data.rsi.values[self.day:self.day+30].tolist()  + \
                          self.data.cci.values[self.day:self.day+30].tolist()  + \
                          self.data.adx.values[self.day:self.day+30].tolist()) 
            
        return self.state
    
    def render(self, mode='human',close=False):
        return self.state
    

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

## Models for our Ensemble Strategy
1. A2C
2. DDPG
3. TD3
4. PPO
5. SAC

In [ ]:
import os
TRAINED_MODEL_DIR = './saved_models'
if not os.path.exists(TRAINED_MODEL_DIR):
    os.mkdir(TRAINED_MODEL_DIR)

In [ ]:
 def train_A2C(env_train, model_name, timesteps=50000):
    start = time.time()
    model = A2C('MlpPolicy', env_train, verbose=1)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{TRAINED_MODEL_DIR}/{model_name}")
    print('Training time (A2C): ', (end - start) / 60, ' minutes')
    return model

def train_DDPG(env_train, model_name, timesteps=10000):
    # add the noise objects for DDPG
    n_actions = env_train.action_space.shape[-1]
    param_noise = None
    action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))
    start = time.time()
    model = DDPG('MlpPolicy', env_train, action_noise=action_noise)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{TRAINED_MODEL_DIR}/{model_name}")
    print('Training time (DDPG): ', (end-start)/60,' minutes')
    return model

def train_TD3(env_train, model_name, timesteps=25000):
    """TD3 model"""

    start = time.time()
    model = TD3('MlpPolicy', env_train, verbose=0)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{TRAINED_MODEL_DIR}/{model_name}")
    print('Training time (TD3): ', (end - start) / 60, ' minutes')
    return model

def train_PPO(env_train, model_name, timesteps=50000):  
    start = time.time()
    model = PPO('MlpPolicy', env_train, ent_coef = 0.005)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{TRAINED_MODEL_DIR}/{model_name}")
    print('Training time (PPO): ', (end - start) / 60, ' minutes')
    return model

def train_SAC(env_train, model_name, timesteps=25000):
    """SAC model"""

    start = time.time()
    model = SAC('MlpPolicy', env_train)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{TRAINED_MODEL_DIR}/{model_name}")
    print('Training time (SAC): ', (end - start) / 60, ' minutes')
    return model


In [ ]:
def DRL_prediction(df,
                   model,
                   name,
                   last_state,
                   iter_num,
                   unique_trade_date,
                   rebalance_window,
                   turbulence_threshold,
                   initial):
    ### make a prediction based on trained model### 

    ## trading env
    trade_data = data_split(df, start=unique_trade_date[iter_num - rebalance_window], end=unique_trade_date[iter_num])
    env_trade = DummyVecEnv([lambda: StockTradeEnv(trade_data,
                                                   turbulence_threshold=turbulence_threshold,
                                                   initial=initial,
                                                   previous_state=last_state,
                                                   model_name=name,
                                                   iteration=iter_num)])
    obs_trade = env_trade.reset()

    for i in range(len(trade_data.index.unique())):
        action, _states = model.predict(obs_trade)
        obs_trade, rewards, dones, info = env_trade.step(action)
        # if i == (len(trade_data.index.unique()) - 2):
            # print(env_test.render())
            # last_state = env_trade.render()
    last_state = env_trade.render()
    df_last_state = pd.DataFrame({'last_state': last_state})
    df_last_state.to_csv('last_state_{}_{}.csv'.format(name, i), index=False)
    return last_state

def DRL_validation(model, test_data, test_env, test_obs) -> None:
    ###validation process###
    for i in range(len(test_data.index.unique())):
        action, _states = model.predict(test_obs)
        test_obs, rewards, dones, info = test_env.step(action)

def get_validation_sharpe(iteration):
    ###Calculate Sharpe ratio based on validation results###
    df_total_value = pd.read_csv('account_value_validation_{}.csv'.format(iteration), index_col=0)
    df_total_value.columns = ['account_value_train']
    df_total_value['daily_return'] = df_total_value.pct_change(1)
    sharpe = (4 ** 0.5) * df_total_value['daily_return'].mean() / \
             df_total_value['daily_return'].std()
    return sharpe

In [ ]:
df

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:04:00+00:00,SPY,186.06,186.06,186.06,186.06,470,0.000000,0.000000,-66.666667,100.000000,0.000000
1,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.000000
2,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.000000
3,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.000000
4,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
57995,2016-03-14 11:07:00+00:00,SPY,180.74,180.74,180.74,180.74,301,-0.018151,44.625136,-72.477064,21.824283,0.776241
57996,2016-03-14 11:08:00+00:00,SPY,180.72,180.76,180.76,180.72,3120,-0.020913,43.718166,-90.126292,26.609318,0.748927
57997,2016-03-14 11:09:00+00:00,SPY,180.73,180.72,180.73,180.71,5420,-0.022040,44.303675,-118.004866,28.934235,0.759701
57998,2016-03-14 11:10:00+00:00,SPY,180.73,180.72,180.73,180.72,9550,-0.022673,44.303675,-116.182573,28.934235,0.757751


In [ ]:
insample_turbulence = train_data
insample_turbulence = insample_turbulence.drop_duplicates(subset=['datadate'])
insample_turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, .90)
insample_turbulence

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
1,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
2,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
3,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
4,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
5,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
34795,2016-02-02 12:59:00+00:00,SPY,172.10,172.03,172.10,172.03,17305,0.029398,56.470884,189.531448,17.517758,0.0
34796,2016-02-02 13:00:00+00:00,SPY,172.15,172.13,172.19,171.83,48448,0.040178,58.161466,159.663866,16.537462,0.0
34797,2016-02-02 13:01:00+00:00,SPY,172.13,172.15,172.16,172.09,7020,0.046571,57.241544,209.414605,16.537462,0.0
34798,2016-02-02 13:02:00+00:00,SPY,172.19,172.10,172.19,172.10,7721,0.055835,59.242193,207.435572,11.022895,0.0


In [ ]:
# historical_turbulence = h1[(h1.datadate<unique_trade_date[3780 - rebalance_window - validation_window]) & (h1.datadate>=(unique_trade_date[3780 - rebalance_window - validation_window-3780]))]

# historical_turbulence = historical_turbulence.drop_duplicates(subset=['datadate'])
# historical_turbulence_mean = np.mean(historical_turbulence.turbulence.values)   

# historical_turbulence

In [ ]:
# unique_trade_date = df[(df.datadate > "2015-12-01 20:44:00+00:00	") & (df.datadate <= "2015-12-02 15:03:00+00:00")].datadate.unique()


# # rebalance_window is the number of months to retrain the model
# # validation_window is the number of months to validation the model and select for trading
# unique_trade_date

In [ ]:
last_state_ensemble = []

ppo_sharpe_list = []
ddpg_sharpe_list = []
a2c_sharpe_list = []
td3_sharpe_list = []
dqn_sharpe_list = []
sac_sharpe_list = []

model_use = []

# based on the analysis of the in-sample data
# turbulence_threshold = 140
# insample_turbulence = df[(df.datadate > "2015-12-01 09:04:00+00:00") & (df.datadate <= "2015-12-01 20:42:00+00:00")]
# insample_turbulence = train_data
# print("********")
# print(insample_turbulence)
# print('*************')
# insample_turbulence = insample_turbulence.drop_duplicates(subset=['datadate'])
# insample_turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, .90)
# insample_turbulence

In [ ]:
from IPython.display import display, HTML

In [ ]:
def data_split(df,start,end):
    data = df[(df.datadate >= start) & (df.datadate < end)]
    data=data.sort_values(['datadate'],ignore_index=True)
    data.index = data.datadate.dt.day.factorize()[0]
#     data.index = data.datadate.dt.day

    return data

In [ ]:
rebalance_window = 30
validation_window = 30

In [ ]:
end=unique_trade_date[0 - rebalance_window - validation_window]
end

Timestamp('2016-03-14 08:42:00+0000', tz='UTC')

In [ ]:
train = data_split(df, start="2015-12-01 09:05:00+00:00", end=unique_trade_date[0 - rebalance_window - validation_window])
train

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.000000
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.000000
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.000000
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.000000
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11,2016-03-14 08:27:00+00:00,SPY,181.11,181.11,181.11,181.11,4800,-0.018454,49.757891,4.029937,0.394144,1.480960
11,2016-03-14 08:28:00+00:00,SPY,181.10,181.10,181.10,181.10,1500,-0.014738,49.296487,-3.647059,1.801078,1.460409
11,2016-03-14 08:33:00+00:00,SPY,181.02,181.02,181.02,181.02,150,-0.018039,45.783016,-80.373406,12.384440,1.318623
11,2016-03-14 08:35:00+00:00,SPY,181.01,181.01,181.01,181.01,325,-0.021219,45.364922,-82.197802,13.628273,1.299572


In [ ]:
# start = '2015-12-01 09:05:00+00:00'
# end = '2015-12-20 09:19:00+00:00'
# data = h1[(h1.datadate >= start) & (h1.datadate < end)]
# data=data.sort_values(['datadate'],ignore_index=True)
# data.index = data.datadate.dt.day.factorize()[0]
# display(data)
# data.datadate.dt.hour
# pd.date_range(start='1/1/2018', end='1/09/2018', freq='M')
# DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
#                '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08'],
#               dtype='datetime64[ns]', freq='D')

In [ ]:

# print("Rebalance Windows: ", rebalance_window)
# print("Unique Trade Date[i]", unique_trade_date[i])
# print("Validation Windows: ", validation_window)
# for i in range(rebalance_window + validation_window, len(unique_trade_date), rebalance_window):
#     print("I is: ", i)    
#     print(unique_trade_date[i])
# #     train = data_split(df, start="2015-12-01 09:05:00+00:00", end=unique_trade_date[i - rebalance_window - validation_window])
#     train = train
#     print("Train is: ")
#     print(len(train))
    

In [ ]:
train_data

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
1,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
2,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
3,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
4,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
5,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
34795,2016-02-02 12:59:00+00:00,SPY,172.10,172.03,172.10,172.03,17305,0.029398,56.470884,189.531448,17.517758,0.0
34796,2016-02-02 13:00:00+00:00,SPY,172.15,172.13,172.19,171.83,48448,0.040178,58.161466,159.663866,16.537462,0.0
34797,2016-02-02 13:01:00+00:00,SPY,172.13,172.15,172.16,172.09,7020,0.046571,57.241544,209.414605,16.537462,0.0
34798,2016-02-02 13:02:00+00:00,SPY,172.19,172.10,172.19,172.10,7721,0.055835,59.242193,207.435572,11.022895,0.0


In [ ]:
train = data_split(df, start="2015-12-01 09:05:00+00:00", end=unique_trade_date[0 - rebalance_window - validation_window])
train

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.000000
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.000000
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.000000
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.000000
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11,2016-03-14 08:27:00+00:00,SPY,181.11,181.11,181.11,181.11,4800,-0.018454,49.757891,4.029937,0.394144,1.480960
11,2016-03-14 08:28:00+00:00,SPY,181.10,181.10,181.10,181.10,1500,-0.014738,49.296487,-3.647059,1.801078,1.460409
11,2016-03-14 08:33:00+00:00,SPY,181.02,181.02,181.02,181.02,150,-0.018039,45.783016,-80.373406,12.384440,1.318623
11,2016-03-14 08:35:00+00:00,SPY,181.01,181.01,181.01,181.01,325,-0.021219,45.364922,-82.197802,13.628273,1.299572


In [ ]:
train = data_split(df, start="2015-12-01 09:05:00+00:00", end=unique_trade_date[ - rebalance_window - validation_window])
train

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.000000
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.000000
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.000000
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.000000
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11,2016-03-14 08:27:00+00:00,SPY,181.11,181.11,181.11,181.11,4800,-0.018454,49.757891,4.029937,0.394144,1.480960
11,2016-03-14 08:28:00+00:00,SPY,181.10,181.10,181.10,181.10,1500,-0.014738,49.296487,-3.647059,1.801078,1.460409
11,2016-03-14 08:33:00+00:00,SPY,181.02,181.02,181.02,181.02,150,-0.018039,45.783016,-80.373406,12.384440,1.318623
11,2016-03-14 08:35:00+00:00,SPY,181.01,181.01,181.01,181.01,325,-0.021219,45.364922,-82.197802,13.628273,1.299572


In [ ]:
# 100 shares per trade
MAX_NORMALIZE = 100
# initial amount of money we have in account
INITIAL_ACC_BAL=1000000
# total number of stocks in  portfolio
TOTAL_STOCKS = 1
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001
REWARD_SCALING = 1e-4

start = time.time()
for i in range(rebalance_window + validation_window, len(unique_trade_date), rebalance_window):
    print("============================================")
    ## initial state is empty
    if i - rebalance_window - validation_window == 0:
        # inital state
        initial = True
    else:
        # previous state
        initial = False

    # Tuning trubulence index based on historical data
    # Turbulence lookback window is one quarter
    historical_turbulence = df[(df.datadate<unique_trade_date[i - rebalance_window - validation_window]) & (df.datadate>=(unique_trade_date[i - rebalance_window - validation_window - 3]))]
#     historical_turbulence = test_data
    historical_turbulence = historical_turbulence.drop_duplicates(subset=['datadate'])
    historical_turbulence_mean = np.mean(historical_turbulence.turbulence.values)   

    if historical_turbulence_mean > insample_turbulence_threshold:
        # if the mean of the historical data is greater than the 90% quantile of insample turbulence data
        # then we assume that the current market is volatile, 
        # therefore we set the 90% quantile of insample turbulence data as the turbulence threshold 
        # meaning the current turbulence can't exceed the 90% quantile of insample turbulence data
        turbulence_threshold = insample_turbulence_threshold
    else:
        # if the mean of the historical data is less than the 90% quantile of insample turbulence data
        # then we tune up the turbulence_threshold, meaning we lower the risk 
        turbulence_threshold = np.quantile(insample_turbulence.turbulence.values, 1)
    print("turbulence_threshold: ", turbulence_threshold)
    
    train = data_split(df, start="2015-12-01 09:05:00+00:00", end=unique_trade_date[i - rebalance_window - validation_window])
    print("I is: ", i)
    print("Train is: ")
    display(train)
    print("Train Shape: ", train.shape)
    print("DONE")
    
    env_train = DummyVecEnv([lambda: StockTrainEnv(train)])
    # env_train = StockTrainEnv(train)
    obs_train = env_train.reset()
    print(f'Obs train \n{obs_train.shape}')
    
    ## validation env
    validation = data_split(df, start=unique_trade_date[i - rebalance_window - validation_window],
                            end=unique_trade_date[i - rebalance_window])
    
    env_val = DummyVecEnv([lambda: StockValidationEnv(validation,
                                                      turbulence_threshold=turbulence_threshold,
                                                      iteration=i)])
    
    # env_val = StockValidationEnv(validation, turbulence_threshold=turbulence_threshold, iteration=i)
    obs_val = env_val.reset()
    print(f'Obs val \n{obs_val.shape}')
    
    print("======Model training from: ", 20090000, "to ",
              unique_trade_date[i - rebalance_window - validation_window])
        # print("training: ",len(data_split(df, start=20090000, end=test.datadate.unique()[i-rebalance_window]) ))
        # print("==============Model Training===========")
    print("======A2C Training========")
    model_a2c = train_A2C(env_train, model_name="A2C_30k_dow_{}".format(i), timesteps=10000)
    print("======A2C Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
          unique_trade_date[i - rebalance_window])
    DRL_validation(model=model_a2c, test_data=validation, test_env=env_val, test_obs=obs_val)
    sharpe_a2c = get_validation_sharpe(i)
    print("A2C Sharpe Ratio: ", sharpe_a2c)

    print("======PPO Training========")
    model_ppo = train_PPO(env_train, model_name="PPO_100k_dow_{}".format(i), timesteps=10000)
    print("======PPO Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
          unique_trade_date[i - rebalance_window])
    DRL_validation(model=model_ppo, test_data=validation, test_env=env_val, test_obs=obs_val)
    sharpe_ppo = get_validation_sharpe(i)
    print("PPO Sharpe Ratio: ", sharpe_ppo)

    print("======TD3 Training========")
    model_td3 = train_TD3(env_train, model_name="TD3_30k_dow_{}".format(i), timesteps=10000)
    print("======TD3 Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
          unique_trade_date[i - rebalance_window])
    DRL_validation(model=model_td3, test_data=validation, test_env=env_val, test_obs=obs_val)
    sharpe_td3 = get_validation_sharpe(i)
    print("TD3 Sharpe Ratio: ", sharpe_td3)

    print("======SAC Training========")
    model_sac = train_SAC(env_train, model_name="SAC_30k_dow_{}".format(i), timesteps=10000)
    print("======SAC Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
          unique_trade_date[i - rebalance_window])
    DRL_validation(model=model_sac, test_data=validation, test_env=env_val, test_obs=obs_val)
    sharpe_sac = get_validation_sharpe(i)
    print("SAC Sharpe Ratio: ", sharpe_sac)

    print("======DDPG Training========")
    model_ddpg = train_DDPG(env_train, model_name="DDPG_10k_dow_{}".format(i), timesteps=10000)
    #model_ddpg = train_TD3(env_train, model_name="DDPG_10k_dow_{}".format(i), timesteps=20000)
    print("======DDPG Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
          unique_trade_date[i - rebalance_window])
    DRL_validation(model=model_ddpg, test_data=validation, test_env=env_val, test_obs=obs_val)
    sharpe_ddpg = get_validation_sharpe(i)

    # print("======DQN Training========")
    # model_ddpg = train_DQN(env_train, model_name="DQN_10k_dow_{}".format(i), timesteps=10000)
    # print("======DQN Validation from: ", unique_trade_date[i - rebalance_window - validation_window], "to ",
    #       unique_trade_date[i - rebalance_window])
    # DRL_validation(model=model_dqn, test_data=validation, test_env=env_val, test_obs=obs_val)
    # sharpe_dqn = get_validation_sharpe(i)


    ppo_sharpe_list.append(sharpe_ppo)
    a2c_sharpe_list.append(sharpe_a2c)
    ddpg_sharpe_list.append(sharpe_ddpg)
    td3_sharpe_list.append(sharpe_td3)
    sac_sharpe_list.append(sharpe_sac)
    # dqn_sharpe_list.append(sharpe_dqn)
    
#     Model Selection based on sharpe ratio
    if (sharpe_ppo >= sharpe_a2c) & (sharpe_ppo >= sharpe_ddpg):
        model_ensemble = model_ppo
        model_use.append('PPO')
    elif (sharpe_a2c > sharpe_ppo) & (sharpe_a2c > sharpe_ddpg):
        model_ensemble = model_a2c
        model_use.append('A2C')
    else:
        model_ensemble = model_ddpg
        model_use.append('DDPG')

    all_sharpes= [ sharpe_a2c  ,sharpe_td3 , sharpe_ddpg, sharpe_ppo ]
    names_sharpes =[
                    (model_a2c,'A2C'),             
                    (model_td3,'TD3'),
                    (model_sac,'SAC'),
                    (model_ddpg,'DDPG'),
                    (model_ppo, 'PPO')         
                    ]
    MAX_SHARPE=all_sharpes.index(max(all_sharpes))
    model_ensemble=names_sharpes[MAX_SHARPE][0]
    model_use.append(names_sharpes[MAX_SHARPE][1])

    print('*'*50)
    print('picked up technique',names_sharpes[MAX_SHARPE][0])
    ############## Training and Validation ends ##############    

    ############## Trading starts ##############    
    print("======Trading from: ", unique_trade_date[i - rebalance_window], "to ", unique_trade_date[i])
    #print("Used Model: ", model_ensemble)
    last_state_ensemble = DRL_prediction(df=df, model=model_ensemble, name="ensemble",
                                         last_state=last_state_ensemble, iter_num=i,
                                         unique_trade_date=unique_trade_date,
                                         rebalance_window=rebalance_window,
                                         turbulence_threshold=turbulence_threshold,
                                         initial=initial)
    
    # print(f'last state ensemble {len(last_state_ensemble)} {last_state_ensemble}')
    # print("============Trading Done============")
    ############## Trading ends ##############    

end = time.time()
print("Ensemble Strategy took: ", (end - start) / 60, " minutes")

turbulence_threshold:  22.48523251674071
I is:  60
Train is: 


,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1,2016-02-02 13:00:00+00:00,SPY,172.15,172.13,172.19,171.83,48448,0.040178,58.161466,159.663866,16.537462,0.0
1,2016-02-02 13:01:00+00:00,SPY,172.13,172.15,172.16,172.09,7020,0.046571,57.241544,209.414605,16.537462,0.0
1,2016-02-02 13:02:00+00:00,SPY,172.19,172.10,172.19,172.10,7721,0.055835,59.242193,207.435572,11.022895,0.0
1,2016-02-02 13:03:00+00:00,SPY,172.17,172.16,172.19,172.16,3199,0.060861,58.301537,192.063186,11.022895,0.0


Train Shape:  (34800, 12)
DONE
Day is:  0
**********
Length is:  1571
Shape is:  (1571, 12)


,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016-02-01 23:54:00+00:00,SPY,172.51,172.51,172.51,172.51,225,-0.063823,35.599604,-160.320856,51.646621,0.0
0,2016-02-01 23:55:00+00:00,SPY,172.54,172.52,172.54,172.52,300,-0.062377,38.014005,-123.017061,40.703761,0.0
0,2016-02-01 23:56:00+00:00,SPY,172.51,172.52,172.52,172.51,200,-0.062926,36.594743,-125.556380,42.190183,0.0
0,2016-02-01 23:57:00+00:00,SPY,172.49,172.55,172.55,172.47,2799,-0.064235,35.676137,-120.225694,47.786723,0.0


**********
Obs train 
(1, 152)
HEYYYY
******************
                    datadate  tic   adjcp    open    high     low  volume  \
0 2016-02-02 13:05:00+00:00  SPY  172.04  172.45  172.45  172.04   49080   
0 2016-02-02 13:06:00+00:00  SPY  172.06  172.08  172.10  172.06    3300   
0 2016-02-02 13:07:00+00:00  SPY  172.04  172.06  172.06  172.04    5425   
0 2016-02-02 13:08:00+00:00  SPY  172.08  172.08  172.09  172.08     850   
0 2016-02-02 13:09:00+00:00  SPY  172.07  172.07  172.08  172.07    9360   
0 2016-02-02 13:10:00+00:00  SPY  172.06  172.06  172.06  172.06     240   
0 2016-02-02 13:11:00+00:00  SPY  172.11  172.07  172.13  172.07   18250   
0 2016-02-02 13:12:00+00:00  SPY  172.08  172.13  172.13  172.08   17700   
0 2016-02-02 13:13:00+00:00  SPY  172.01  172.07  172.07  172.01    7950   
0 2016-02-02 13:14:00+00:00  SPY  171.95  172.02  172.02  171.95   14495   
0 2016-02-02 13:15:00+00:00  SPY  171.97  171.95  171.99  171.92   16220   
0 2016-02-02 13:16:00+00:00  S

...

**********
Obs train 
(1, 152)
HEYYYY
******************
                    datadate  tic   adjcp    open    high     low   volume  \
0 2016-02-02 16:05:00+00:00  SPY  170.70  170.65  170.72  170.63   115575   
0 2016-02-02 16:06:00+00:00  SPY  170.74  170.71  170.76  170.69    64354   
0 2016-02-02 16:07:00+00:00  SPY  170.83  170.74  170.84  170.73   129326   
0 2016-02-02 16:08:00+00:00  SPY  170.89  170.83  170.90  170.81   282123   
0 2016-02-02 16:09:00+00:00  SPY  170.84  170.89  170.92  170.84   180918   
0 2016-02-02 16:10:00+00:00  SPY  170.81  170.84  170.91  170.80   157194   
0 2016-02-02 16:11:00+00:00  SPY  170.83  170.82  170.86  170.77   119599   
0 2016-02-02 16:12:00+00:00  SPY  170.99  170.83  171.00  170.80   189567   
0 2016-02-02 16:13:00+00:00  SPY  171.07  170.99  171.09  170.95   561855   
0 2016-02-02 16:14:00+00:00  SPY  171.20  171.06  171.22  171.05   340228   
0 2016-02-02 16:15:00+00:00  SPY  171.19  171.20  171.23  171.09   557744   
0 2016-02-02 16:16

,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1,2016-02-02 16:39:00+00:00,SPY,171.30,171.36,171.37,171.29,1669171,0.103597,57.500155,101.822729,35.391905,0.0
1,2016-02-02 16:40:00+00:00,SPY,171.23,171.31,171.32,171.21,1775875,0.097195,55.504860,75.371341,23.078595,0.0
1,2016-02-02 16:41:00+00:00,SPY,171.25,171.24,171.32,171.22,108341,0.092667,55.956585,75.431212,23.078595,0.0
1,2016-02-02 16:42:00+00:00,SPY,171.20,171.25,171.31,171.19,178926,0.084075,54.524989,61.075269,18.462020,0.0


Train Shape:  (35010, 12)
DONE
Day is:  0
**********
Length is:  1571
Shape is:  (1571, 12)


,datadate,tic,adjcp,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,2015-12-01 09:05:00+00:00,SPY,185.96,185.99,185.99,185.96,1160,-0.002244,0.000000,-66.666667,100.000000,0.0
0,2015-12-01 09:06:00+00:00,SPY,186.05,186.05,186.05,186.05,100,-0.000068,48.214286,41.176471,21.848739,0.0
0,2015-12-01 09:18:00+00:00,SPY,186.07,186.07,186.07,186.07,1000,0.001645,53.381375,64.197531,6.956632,0.0
0,2015-12-01 09:19:00+00:00,SPY,186.06,186.06,186.06,186.05,800,0.002144,50.761597,35.825545,17.707840,0.0
0,2015-12-01 09:20:00+00:00,SPY,186.05,186.06,186.06,186.05,520,0.001976,48.309000,27.272727,17.707840,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016-02-01 23:54:00+00:00,SPY,172.51,172.51,172.51,172.51,225,-0.063823,35.599604,-160.320856,51.646621,0.0
0,2016-02-01 23:55:00+00:00,SPY,172.54,172.52,172.54,172.52,300,-0.062377,38.014005,-123.017061,40.703761,0.0
0,2016-02-01 23:56:00+00:00,SPY,172.51,172.52,172.52,172.51,200,-0.062926,36.594743,-125.556380,42.190183,0.0
0,2016-02-01 23:57:00+00:00,SPY,172.49,172.55,172.55,172.47,2799,-0.064235,35.676137,-120.225694,47.786723,0.0


**********
Obs train 
(1, 152)
HEYYYY
******************
                    datadate  tic   adjcp    open    high     low   volume  \
0 2016-02-02 16:44:00+00:00  SPY  171.06  171.11  171.14  171.03   258850   
0 2016-02-02 16:45:00+00:00  SPY  170.97  171.06  171.10  170.97   765924   
0 2016-02-02 16:46:00+00:00  SPY  171.02  170.97  171.04  170.97   113607   
0 2016-02-02 16:47:00+00:00  SPY  170.97  171.02  171.02  170.95   130540   
0 2016-02-02 16:48:00+00:00  SPY  170.83  170.97  170.97  170.82   333703   
0 2016-02-02 16:49:00+00:00  SPY  170.84  170.83  170.89  170.81   176084   
0 2016-02-02 16:50:00+00:00  SPY  170.84  170.84  170.85  170.76   240289   
0 2016-02-02 16:51:00+00:00  SPY  170.88  170.84  170.89  170.84   177211   
0 2016-02-02 16:52:00+00:00  SPY  170.92  170.87  170.93  170.83   208070   
0 2016-02-02 16:53:00+00:00  SPY  170.91  170.92  170.94  170.87   241140   
0 2016-02-02 16:54:00+00:00  SPY  170.86  170.90  170.90  170.85    69028   
0 2016-02-02 16:55

In [ ]:
ppo_sharpe_list

NameError: ignored

In [ ]:
 #!pip freeze >> requirements.txt # this gives the environment 